In [2]:
from unittest.mock import Mock

mock = Mock()
json = mock 

In [4]:
# Unlike the real dumps(), this mocked method requires no arguments. In fact, it will accept any arguments that you pass to it.

json.dumps()

# The return value of dumps() is also a Mock. The capability of Mock to recursively define other mocks allows for you to use mocks in complex situations:

<Mock name='mock.dumps()' id='4457885904'>

In [5]:
json.loads('{"k": "v"}').get('k')

<Mock name='mock.loads().get()' id='4457694976'>

In [6]:
json.loads.assert_called()

In [7]:
json.loads.assert_called_once()

In [8]:
# let's make it give an error cause it is waiting that the call happened with '{"k": "v"}'
json.loads.assert_called_with('{"key": "value"}')

AssertionError: expected call not found.
Expected: loads('{"key": "value"}')
Actual: loads('{"k": "v"}')

In [11]:
json.loads('{"k": "v"}').get('k')

<Mock name='mock.loads().get()' id='4457694976'>

In [12]:
json.loads.assert_called_once_with('{"k": "v"}')

AssertionError: Expected 'loads' to be called once. Called 2 times.
Calls: [call('{"k": "v"}'), call().get('k'), call('{"k": "v"}'), call().get('k')].

In [15]:
# positional argument
json.loads(s='{"key": "value"}')

# key word argument
json.loads.assert_called_with('{"key": "value"}')

AssertionError: expected call not found.
Expected: loads('{"key": "value"}')
Actual: loads(s='{"key": "value"}')

In [16]:
# how many time your application called loads
json.loads.call_count

5

In [17]:
# last arguments given
json.loads.call_args

call(s='{"key": "value"}')

In [18]:
# list of every call of loads method
json.loads.call_args_list

[call('{"k": "v"}'),
 call('{"k": "v"}'),
 call(s='{"key": "value"}'),
 call(s='{"key": "value"}'),
 call(s='{"key": "value"}')]

In [19]:
# list of every call of json object
json.method_calls

[call.do_something(),
 call.dumps(),
 call.loads('{"k": "v"}'),
 call.loads('{"k": "v"}'),
 call.loads(s='{"key": "value"}'),
 call.loads(s='{"key": "value"}'),
 call.loads(s='{"key": "value"}')]

# Return value role 


One reason to use mocks is to control your code’s behavior during tests. One way to do this is to specify a function’s return value. Let’s use an example to see how this works.



In [ ]:
# create a file named my_calendar.py

# paste this code in it 

from datetime import datetime

def is_weekday():
    today = datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    # sunday & saturday are off
    return (0 <= today.weekday() < 5)

# Test if today is a weekday
assert is_weekday()

In [20]:

import datetime
from unittest.mock import Mock

# Save a couple of test days
tuesday = datetime.datetime(year=2019, month=1, day=1)
saturday = datetime.datetime(year=2019, month=1, day=5)

# Mock datetime to control today's date
datetime = Mock()

def is_weekday():
    today = datetime.datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return (0 <= today.weekday() < 5)



# Mock .today() to return Tuesday
datetime.datetime.today.return_value = tuesday
# Test Tuesday is a weekday
assert is_weekday()
# Mock .today() to return Saturday
datetime.datetime.today.return_value = saturday
# Test Saturday is not a weekday
assert not is_weekday()

In [ ]:
...

In [ ]:
import requests
import unittest
from unittest.mock import Mock

# Mock requests to control its behavior
requests = Mock()

def get_holidays():
    r = requests.get('http://localhost/api/holidays')
    if r.status_code == 200:
        return r.json()
    return None

class TestCalendar(unittest.TestCase):
    def log_request(self, url):
        # Log a fake request for test output purposes
        print(f'Making a request to {url}.')
        print('Request received!')

        # Create a new Mock to imitate a Response
        response_mock = Mock()
        response_mock.status_code = 200
        response_mock.json.return_value = {
            '12/25': 'Christmas',
            '7/4': 'Independence Day',
        }
        return response_mock

    def test_get_holidays_logging(self):
        # Test a successful, logged request
        requests.get.side_effect = self.log_request
        assert get_holidays()['12/25'] == 'Christmas'

if __name__ == '__main__':
    unittest.main()

In [23]:
import unittest
from requests.exceptions import Timeout
from unittest.mock import Mock

# Mock requests to control its behavior
requests = Mock()

def get_holidays():
    r = requests.get('http://localhost/api/holidays')
    if r.status_code == 200:
        return r.json()
    return None

class TestCalendar(unittest.TestCase):
    def test_get_holidays_retry(self):
        # Create a new Mock to imitate a Response
        response_mock = Mock()
        response_mock.status_code = 200
        response_mock.json.return_value = {
            '12/25': 'Christmas',
            '7/4': 'Independence Day',
        }
        # Set the side effect of .get()
        requests.get.side_effect = [Timeout, response_mock]
        # Test that the first request raises a Timeout
        with self.assertRaises(Timeout):
            get_holidays()
        # Now retry, expecting a successful response
        assert get_holidays()['12/25'] == 'Christmas'
        # Finally, assert .get() was called twice
        assert requests.get.call_count == 2


if __name__ == '__main__':
    unittest.main()

In [27]:
mock = Mock(side_effect=Exception)

In [29]:
mock()

Exception: 

In [30]:
# you can also set name 

mock = Mock(name="mock python")

In [32]:
mock

<Mock name='mock python ' id='4482145008'>

In [33]:
mock = Mock()
mock.configure_mock(return_value=True)

In [34]:
mock()

True

In [35]:
# Verbose, old Mock
response_mock = Mock()
response_mock.json.return_value = {
    '12/25': 'Christmas',
    '7/4': 'Independence Day',
}

# Shiny, new .configure_mock()
holidays = {'12/25': 'Christmas', '7/4': 'Independence Day'}
response_mock = Mock(**{'json.return_value': holidays})

# Patch using 

Referencing where you want to mock object does make a difference Notice

In [2]:
import my_calendar
from unittest.mock import patch

with patch('my_calendar.is_weekday'):
    print(my_calendar.is_weekday())
    
    

<MagicMock name='is_weekday()' id='4506189104'>


In [3]:
from unittest.mock import patch
from my_calendar import is_weekday

with patch('my_calendar.is_weekday'):
    print(is_weekday())

False


In [4]:
from unittest.mock import patch
from my_calendar import is_weekday

with patch('__main__.is_weekday'):
    print(is_weekday())

<MagicMock name='is_weekday()' id='4506205200'>


In [1]:
from unittest.mock import Mock

calendar = Mock(spec=['is_weekday', 'get_holidays'])
calendar.is_weekday()

<Mock name='mock.is_weekday()' id='4600091648'>

In [2]:
calendar.create_event()

AttributeError: Mock object has no attribute 'create_event'